In [2]:
%%javascript
    /*
        WiggiMX:feel free to use it just give us the credit with a short mention :)
    */
    $("#homeButton").remove()
    $('body').append('<a href="#'+$("h1,h2,h3:eq(0)").attr("id")+'" style="position: fixed; bottom: 10px; right: 10px;" type="button" class="btn btn-success btn-circle btn-lg"><i class="glyphicon glyphicon-link" id="homeButton"></i></a>');
    $("#MainMenu").remove()
    var menu = 
      '<div id="MainMenu" style="position: fixed; top: 120px; right: 10px; z-index:6; max-height: 500px;">'+
        '<div class="list-group panel" >'+
          '<a href="#collapseMenu" class="list-group-item list-group-item-success" data-toggle="collapse" data-parent="#MainMenu"><img width="20px" src="/kernelspecs/python3/logo-64x64.png"/><i class="fa fa-caret-down"></i></a>'+
          '<div class="collapse" id="collapseMenu" style="overflow-y: overlay; max-height: 500px;">'+
          '</div>'+
        '</div>'+
      '</div>'
   
    var parent = $(menu)
    var arrayIds = []
    $("h1,h2,h3").attr("id",function(id,Value){
        if(Value != ""){
            var content = (Value.replace(new RegExp('-', 'g'), ' '));
            content = "&nbsp;".repeat(parseInt($(this).prop("tagName")[1])*2-1) + content;
            $(parent).find("#collapseMenu").append('<a href="#'+Value+'" style="position:relative;" class="list-group-item" data-parent="#SubMenu1">'+content+'</a>');
        }
    })
$('body').append(parent)

<IPython.core.display.Javascript object>

# RDDs

Los RDDs o conjuntos de datos resilientes (**resilent distributed data sets**) se consideran la estructura de datos principal en Spark (existen otras como los data frames).Son listas distribuidas resilientes a errores.

Los RDDs son distribuidos porque están particionados en los workers.

Se pueden pensar como una lista a la que uno no tiene acceso directamente. 

<img src='images/rdd.png' width="50%" height="50%"/>

<center>*RDD separado en 5 particiones, algunas de ellas residen en el mismo worker*</center>

### Operaciones en RDDs: Transformaciones y acciones

- **Transformaciones**: Son operaciones que dan como resultado un nuevo RDD. Son *lazy* ya que no se ejecutan sino hasta que tiene lugar una acción.
- **Acciones**: Son opreaciones que dan como resultado un número. Al ejecutarlas desencadenan la serie de transformaciones previa a poder calcularla. 


### Características

<div class="alert alert-success alert-dismissable" id="def2">
<a href="#" class="close" data-dismiss="alert" aria-label="close">&times;</a>
 <strong>Los RDD's son:</strong>  inmutables, particionados, tolerantes a errores, *lazy*
</div>

**Inmutable**: cada partición de un RDD es una división de datos inmutable, que no puede modificarse. En las transformaciones se crean nuevos RDDs no se transforma el original.

**Particionado**: Los RDDs están divididos en partes a través de los workers

**Tolerante a errores**: Spark va guardando un mapa de las transformaciones a manera de un gráfico de RDDs. En caso de que perdamos alguna partición del RDD, podemos volver a efectuar la transformación en esa partición para lograr computar de nuevo la transformación.

***Lazy***: Las transformaciones sólo se llevaran a cabo al correr una acción.


### Gráfico de RDDs
El gráfico de RDDs define el linaje de los RDDs. Por ejemplo:

```Python
firstRDD = spark.textFile("hdfs://...")
secondRDD = firstRDD.filter(someFunction);
thirdRDD = secondRDD.map(someFunction);
result = thirdRDD.count()
```

![Rdd Graph](./images/RDD_graph.png)
Example taken from ([saurzcode](https://saurzcode.in/2015/10/what-is-rdd-in-spark-and-why-do-we-need-it/))

Una vez que inicializamos el SparkContext hay tres formas generales de crear un RDD:

a) Leer un RDD desde archivo

In [11]:
import pandas as pd
from pyspark.sql import SparkSession
from pyspark.sql import functions as F
spark = SparkSession \
       .builder.master("local[*]") \
       .appName("RDD") \
       .getOrCreate()

sc = spark.sparkContext

In [12]:
textFile = sc.textFile("input.txt")

b) Usar parallelize ya sea para un archivo o arreglo 

In [13]:
fileString = open('data/Amounts.txt','r').read().split('\n')
numbersStrRDD = sc.parallelize(fileString)

In [14]:
type(numbersStrRDD)

pyspark.rdd.RDD

In [15]:
miFirstRDD = sc.parallelize([1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25])
type(miFirstRDD)

pyspark.rdd.RDD

<div class="alert alert-success alert-dismissable" id="def2">
<a href="#" class="close" data-dismiss="alert" aria-label="close">&times;</a>
 Dado que los RDDs no tienen un orden definido, las operaciones en ellos <strong>no deben depender del orden de los elementos en las listas ni del orden en el que se efectúan las operaciones</strong></div>
 
 ## Más sobre operaciones con RDDs
 
- **Transformaciones**: Son operaciones que dan como resultado un nuevo RDD. Son *lazy* ya que no se ejecutan sino hasta que tiene lugar una acción.
- **Acciones**: Son opreaciones que dan como resultado un número. Al ejecutarlas desencadenan la serie de transformaciones previa a poder calcularla. 

![OperacionesRDDs](./images/RddOperations.png)

* **collect()**: Esta operación regresa a la memoria local los elmentos que están distribuidos en el clúster como una lista secuencial.

<div class="alert alert-success alert-dismissable" id="def2">
<a href="#" class="close" data-dismiss="alert" aria-label="close">&times;</a>
 <strong>Si tenemos muchos datos (masivos) esta operación puede matar nuestra memoria x_x</strong></div>

* **take(número n chiquito)**: Está función hace un collect de n elelemntos arbitrarios del conjunto de datos y los acomoda en memoria local. No es determinista por lo que puede ser que se obtengan distintos sets de datos cada vez.

**Ejemplo**: Obteniendo elementos de un RDD a la memoria local.

In [16]:
print('Collect: '+str(miFirstRDD.collect()))
print('Take: '+str(miFirstRDD.take(5)))

Collect: [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25]
Take: [1, 2, 3, 4, 5]


### Map 
---

map es un tipo de  **transformación** que crea un nuevo data set a partic de uno ya existente por medio de un **mapeo uno a uno**.

#### Nota: Operador Lambda

En python y scala las operaciones **map**, **reduce** y **filter**se aplican a través de expresiones a través del operador lambda. El operador lambda o función lambda nos permite crear funciones anónimas (sólo necesarias donde se crearon).

Reordemos la sintaxis general de una función lambda:

**lambda** lista de argumentos: **expression**.

Más info del operador lambda: [http://www.python-course.eu/lambda.php]

** Example: Map Example**

In [17]:
numbersStrRDD.take(5)

['594.38', '281.37', '144.85', '20.99', '797.40']

In [18]:
# Casting the elements of the RDD into a distributes way
floatsRDD = numbersStrRDD.map(lambda x: float(x))
floatsRDD.take(5)

[594.38, 281.37, 144.85, 20.99, 797.4]

### Ejercicios

1.¿Cómo calcularías en el RDD "l" la suma de los cuadrados de los elementos?


In [19]:
l=sc.parallelize([1,2,3,4,5,6])

Si tuviera python

In [20]:
p = [1,2,3,4,5,6]

In [22]:
import functools
functools.reduce(lambda x,y: x+y, map(lambda i:i*i,p))

91

En spark, ¿cómo sería si consideramos que map y reduce se aplican de la siguiente manera?
```Python
rdd.map(lambda function)
rdd.reduce(lambda function)
```

**pista**: en Spark las operaciones suelen seriarse de izquierda derecha usand el punto

In [ ]:
l.map(lambda i:i*i).reduce(lambda x,y: x+y)

2.Recordando map y reduce y cómo se efectuan en python, dado el siguiente RDD, ¿cuál de las siguientes operaciones computa el promedio?

In [ ]:
l.reduce(lambda x,y: x+y)/l.map(lambda x:1).reduce(lambda x,y: x+y)

In [ ]:
l.reduce(lambda x,y: x+y)/l.map(lambda x:x).reduce(lambda x,y: x+y)

In [ ]:
l.reduce(lambda x,y: x+y)/l.reduce(lambda x,y: x+y).map(lambda x:x)

In [ ]:
l.reduce(lambda x,y: x+y)/l.reduce(lambda x,y: 1+1)

cuidado al reducir operaciones...

In [ ]:
C=sc.parallelize([1,1,2])
C.reduce(lambda x,y: x*x+y*y)

In [ ]:
C=sc.parallelize([1,1,2])
C.map(lambda x: x*x).reduce(lambda x,y:x+y)

## Con las operaciones se puede obtener información sobre un RDD sin necesidad de desplegarlo todo o hacer un collect
### count
Para saber cuántos elementos hay en un RDD, pero si el set de datos es muy grande puede ser problemático

In [ ]:
n=1000000
B=sc.parallelize([1,2,3,4]*int(n/4))
B.count()

### primeros elementos (poquitos)

In [ ]:
# obtener los primeros elementos de un RDD
print('first element=',B.first())
print('first 5 elements = ',B.take(5))

### Saber si hay entradas en el RDD

In [ ]:
B.isEmpty()

### Conocer un aproximado del número de elementos en el RDD

In [ ]:
B.countApprox(100,confidence=0.95)

### Muestreo

In [ ]:
m=5
# First argument: with or without replacement , second argument: with what probability we are going to get each element
print('sample=',B.sample(False,m/n).collect())

## Filtrado
Regresa un nuevo RDD en el que una condición se cumple

In [ ]:
print('El número de elementos en B que son  > 3 =',B.filter(lambda n: n > 3).count())

## distinct
Quitar elementos duplicados en el RDD

In [ ]:
# Remover elementos duplicados en DuplicateRDD
DuplicateRDD = sc.parallelize([1,1,2,2,3,3])
print('DuplicateRDD=',DuplicateRDD.collect())
print('DistinctRDD = ',DuplicateRDD.distinct().collect())

## flatmap
Hace una lista con todos los elementos

In [23]:
text=["you are my sunshine","my only sunshine"]
text_file = sc.parallelize(text)
# mapea cada línea en el texto a una lista de palabras
print('map:',text_file.map(lambda line: line.split(" ")).collect())
# crea una sola lista de palabras al combinar las palabras de todas las líneas
print('flatmap:',text_file.flatMap(lambda line: line.split(" ")).collect())

map: [['you', 'are', 'my', 'sunshine'], ['my', 'only', 'sunshine']]
flatmap: ['you', 'are', 'my', 'sunshine', 'my', 'only', 'sunshine']


## Operaciones de conjuntos
union, intersection, subtraction, cartesian

In [24]:
rdd1 = sc.parallelize([1, 1, 2, 3])
rdd2 = sc.parallelize([1, 3, 4, 5])

### union
Regresa la unión de un RDD con otro.
Se permiten las repeticiones. Los RDDs son como bolsas más que conjuntos.
Para convertir el resultado en un conjunto use .distinct()

In [25]:
rdd2=sc.parallelize(['a','b',1])
print('rdd1=',rdd1.collect())
print('rdd2=',rdd2.collect())
print('union as bags =',rdd1.union(rdd2).collect())
print('union as sets =',rdd1.union(rdd2).distinct().collect())

rdd1= [1, 1, 2, 3]
rdd2= ['a', 'b', 1]
union as bags = [1, 1, 2, 3, 'a', 'b', 1]
union as sets = [2, 'b', 1, 3, 'a']


### intersection
Intersección. Selecciona los elementos en común entre dos RDDs. La intersección no contiene elementos repetidos, ya que efectúa un distinct internamente.

In [26]:
rdd2=sc.parallelize([1,1,2,5])
print('rdd1=',rdd1.collect())
print('rdd2=',rdd2.collect())
print('intersection=',rdd1.intersection(rdd2).collect())

rdd1= [1, 1, 2, 3]
rdd2= [1, 1, 2, 5]
intersection= [2, 1]


### subtract
Regresa todos los elementos en un RDD que no están contenidos en el otro.

In [27]:
print('rdd1=',rdd1.collect())
print('rdd2=',rdd2.collect())
print('rdd1.subtract(rdd2)=',rdd1.subtract(rdd2).collect())

rdd1= [1, 1, 2, 3]
rdd2= [1, 1, 2, 5]
rdd1.subtract(rdd2)= [3]


### cartesian
Regresa un RDD con todos los pareds de elementos (a,b), donde a está en el RDD del que se manda a llamar el método cartesian y b está en el otro.

In [28]:
rdd1=sc.parallelize([1,1,2])
rdd2=sc.parallelize(['a','b'])
print('rdd1=',rdd1.collect())
print('rdd2=',rdd2.collect())
print('rdd1.cartesian(rdd2)=\n',rdd1.cartesian(rdd2).collect())

rdd1= [1, 1, 2]
rdd2= ['a', 'b']
rdd1.cartesian(rdd2)=
 [(1, 'a'), (1, 'b'), (1, 'a'), (1, 'b'), (2, 'a'), (2, 'b')]


# Fuente
<a src="https://github.com/LiberPH">PhD Libertad</a>